<a href="https://colab.research.google.com/github/YY2mk/KT_aivle/blob/master/DX_%EB%AF%B8%EB%8B%88%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8%205%EC%B0%A8_4%EC%9D%BC%EC%B0%A8%20%EC%8B%A4%EC%8A%B5%EC%9E%90%EB%A3%8CFace_Recognition_%EC%97%90%EC%9D%B4%EB%B8%94%EB%9F%AC%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Face Recognition
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리해야 합니다.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습과 추론을 진행하세요.
    3. Extra
        - 조원들의 얼굴 이미지를 수집한 후, 조원들을 구분하는 모델을 만들어보세요.

### Google Drive 연동

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 데이터셋 압축 해제
- Face_Recognition.zip : 이미지 데이터 압축 파일

In [ ]:
import os, glob, shutil, zipfile
from tqdm import tqdm

In [ ]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[-20:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./{file_name[-20:-4]}/')

In [ ]:
dataset_path = '/content/drive/MyDrive/program/miniPJT_5/DX_미니프로젝트 5차_4일차/Face_Recognition.zip'

In [ ]:
dataset_extract(dataset_path)

Extracting: 100%|██████████| 11960/11960 [00:06<00:00, 1741.08files/s]


### 데이터셋 살펴보기

In [ ]:
# 데이터셋 폴더 내에 있는 모든 파일의 경로 가져오기
dataset_folder = '/content/Face_Recognition/train' # 실제 데이터셋 폴더 경로
num_files = len(os.listdir(dataset_folder))
print("데이터셋 이미지파일의 개수:", num_files//2)

데이터셋 이미지파일의 개수: 5214


### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식이 있습니다.
- 해당 형식에 맞춰 폴더를 만드세요.

In [ ]:
dataset_img = '/content/Face_Recognition/train/images/'
dataset_label = '/content/Face_Recognition/train/labels/'
datavlid_img = '/content/Face_Recognition/valid/images/'
datavlid_label = '/content/Face_Recognition/valid/labels/'

data_path = [dataset_img, dataset_label, datavlid_img, datavlid_label]

# 폴더 생성
for path in data_path:
  os.makedirs(path)

### 폴더에 맞게 파일 이동
- 이전 단계에서 만든 하위 폴더에 이미지 파일과 텍스트 파일을 이동시키세요.

In [ ]:
dataset= '/content/Face_Recognition/train/'
datavlid = '/content/Face_Recognition/valid/'

#glob.glob('/content/Face_Recognition/train/*jpg') # jpg만 가져
#shutil.move(os.path.join(dataset, img), os.path.join(dataset_img, img)) # 기존의 위치(src)에서 옮길 위치(dst)를 설정
# train-이미지
mg_files_jpg = glob.glob(os.path.join(dataset, '*.jpg'))  # train 폴더 안의 모든 .jpg 파일 경로 가져오기
mg_files_txt = glob.glob(os.path.join(dataset, '*.txt'))  # train 폴더 안의 모든 .txt 파일 경로 가져오기
mg_files_jpg_valid = glob.glob(os.path.join(datavlid, '*.jpg'))  # valid 폴더 안의 모든 .jpg 파일 경로 가져오기
mg_files_txt_valid = glob.glob(os.path.join(datavlid, '*.txt'))  # valid 폴더 안의 모든 .txt 파일 경로 가져오기

# dataset_img로 각 .jpg 파일을 이동
for img_file in mg_files_jpg:
    shutil.move(img_file, dataset_img)

# dataset_label로 각 .txt 파일을 이동
for img_file in mg_files_txt:
    shutil.move(img_file, dataset_label)

# valid_img로 각 .jpg 파일을 이동
for img_file in mg_files_jpg_valid:
    shutil.move(img_file, datavlid_img)

# valid_label로 각 .txt 파일을 이동
for img_file in mg_files_txt_valid:
    shutil.move(img_file, datavlid_label)

### YOLO 모델에 적용할 YAML 생성하기
- 클래스는 총 13개입니다. 클래스 순서에 주의하세요.
- 'Barack Obama', 'Che Guevara', 'Cristiano Ronaldo', 'David Beckham', 'Donald Trump', 'Elon Musk', 'Joe Biden', 'Lionel Messi', 'Mark Zuckerberg', 'Oprah Winfrey', 'Steve Harvey', 'Steve Jobs', 'Zaha Hadid'

In [ ]:
import yaml

In [ ]:
## yaml 파일 읽고 수정 필요
# yaml 형식에 맞는 데이터
data = {
    'train': './train/images',
    'val': './valid/images',
    'nc': 13,
    'names': [
        'Barack Obama',
        'Che Guevara',
        'Cristiano Ronaldo',
        'David Beckham',
        'Donald Trump',
        'Elon Musk',
        'Joe Biden',
        'Lionel Messi',
        'Mark Zuckerberg',
        'Oprah Winfrey',
        'Steve Harvey',
        'Steve Jobs',
        'Zaha Hadid'
    ]
}

# yaml 파일 생성
with open('/content/Face_Recognition/famous_human.yaml', 'w') as f:
    yaml.dump(data, f) # data를 yaml 형식으로 변환하여 파일에 씀

### YOLO v8 모델 사용
- yaml 파일의 경로 설정에 주의하세요.

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 15.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
from ultralytics import settings, YOLO

In [ ]:
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/datasets',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [ ]:
settings['datasets_dir'] = '/content/'

In [ ]:
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

model = YOLO(model='yolov8s.pt', task='detect') # task : detectt, segment, classify, pose
                                                # model : yolov8n.pt, yolov8s.pt, yolov8m.pt yolov8l.pt, yolov8x.pt

es = EarlyStopping(monitor='val_loss', patience=5)
mc = ModelCheckpoint('best.pt', monitor='val_loss', save_best_only=True)

100%|██████████| 21.5M/21.5M [00:00<00:00, 343MB/s]


In [ ]:
model.train(model = '/content/yolov8s.pt',
            data='/content/Face_Recognition/famous_human.yaml',
            epochs=100,
            verbose=True,
            patience=15,       # mAP[:50] : mAP[50:95] = 1 : 9
            seed=2024,
            pretrained=True,
            hsv_h=0,
            hsv_s=0,
            hsv_v=0,
            translate=0,
            scale=0,
            fliplr=0,
            mosaic=0,
            erasing=0,
            crop_fraction=0
            )

Ultralytics YOLOv8.2.5 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/yolov8s.pt, data=/content/Face_Recognition/famous_human.yaml, epochs=100, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels

100%|██████████| 755k/755k [00:00<00:00, 137MB/s]

Overriding model.yaml nc=80 with nc=13

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

 22        [15, 18, 21]  1   2121079  ultralytics.nn.modules.head.Detect           [13, [128, 256, 512]]         
Model summary: 225 layers, 11140631 parameters, 11140615 gradients, 28.7 GFLOPs

Transferred 349/355 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir runs/detect/train', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


100%|██████████| 6.23M/6.23M [00:00<00:00, 305MB/s]


...

                   all        765        765     0.0166     0.0103    0.00384   0.000486
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 15, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



30 epochs completed in 0.709 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.5 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11130615 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:06<00:00,  3.57it/s]


                   all        765        765      0.146       0.12     0.0775     0.0137
          Barack Obama        765         56      0.174      0.339      0.147     0.0275
           Che Guevara        765         38      0.112     0.0789     0.0684    0.00841
     Cristiano Ronaldo        765         52      0.144      0.269       0.14     0.0296
         David Beckham        765         75      0.193       0.12     0.0921      0.016
          Donald Trump        765         74     0.0555     0.0405     0.0145    0.00256
             Elon Musk        765         74      0.194     0.0946      0.048    0.00887
             Joe Biden        765         80      0.261      0.167      0.105     0.0168
          Lionel Messi        765         69      0.293      0.159      0.162     0.0313
       Mark Zuckerberg        765         39          0          0    0.00564    0.00128
         Oprah Winfrey        765         49      0.216      0.096      0.101      0.013
          Steve Harve

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b1936819b70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.04504

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF=8"

In [ ]:
result = model.predict(source='/content/Face_Recognition/valid/images',
                       conf=0.5,          # confidence score
                       iou=0.6,           # IoU가 0.7 이상 넘기면 같은 object를 가리키는 것이라고 판단
                       save=True,         # 예측된 이미지/동영상 저장 유무 설정
                       line_width=2       # 그려지는 박스이 두께 설정
                       )

In [ ]:
for r in result :
    boxes = r.boxes

## Extra) 조원들의 얼굴을 구분하는지 테스트 해보자!
1. 데이터 수집 : 조원들의 얼굴 사진을 수집한다! -> 이미지 파일
2. Annotation 작업 : 얼굴 부위에 대해 라벨링을 한다! -> txt 파일
3. 본인 구글 드라이브에 업로드 및 Colab과의 연동!
4. YAML 파일 생성 : 조원들의 사진이 있는 경로와 클래스 수, 클래스 이름 정보가 있는 YAML을 새로 생성!
5. 추가 학습 : 위의 단계에서 생성된 최적의 가중치를 가진 모델을 불러와 추가 학습을 시킨다!
6. 테스트 : 조원들을 잘 인식하는지 predict 해본다!

### YAML 새로 생성하기
- 클래스의 수는 조원의 수입니다.
- Annotation 작업 과정에서 정해진 클래스의 순서 규칙을 따라주세요.

In [2]:
import os, glob, shutil, zipfile
from tqdm import tqdm

In [3]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[-20:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./{file_name[-20:-4]}/')

In [4]:
dataset_path = '/content/drive/MyDrive/program/miniPJT_5/DX_미니프로젝트 5차_4일차/16조.zip'


In [5]:
dataset_extract(dataset_path)

Extracting: 100%|██████████| 2348/2348 [00:34<00:00, 67.21files/s]  


In [6]:
# 데이터셋 폴더 내에 있는 모든 파일의 경로 가져오기
dataset_folder = '/content/ᅳ 5차_4일차/16조/16┴╢/train/images' # 실제 데이터셋 폴더 경로
num_files = len(os.listdir(dataset_folder))
print("데이터셋 이미지파일의 개수:", num_files)

데이터셋 이미지파일의 개수: 941


In [7]:
# 데이터셋 폴더 내에 있는 모든 파일의 경로 가져오기
dataset_folder = '/content/ᅳ 5차_4일차/16조/16┴╢/test/images' # 실제 데이터셋 폴더 경로
num_files = len(os.listdir(dataset_folder))
print("데이터셋 이미지파일의 개수:", num_files)

데이터셋 이미지파일의 개수: 230


In [8]:
import yaml

In [9]:
## yaml 파일 읽고 수정 필요
# yaml 형식에 맞는 데이터
data = {
    'train': './train/images',
    'val': './test/images',
    'nc': 6,
    'names': [
        'SIJ',
        'LCH',
        'KJE',
        'KHS',
        'YMS',
        'YBH'
    ]
}

# yaml 파일 생성
with open('/content/ᅳ 5차_4일차/16조/16┴╢/16_team.yaml', 'w') as f:
    yaml.dump(data, f) # data를 yaml 형식으로 변환하여 파일에 씀

In [10]:
!pip install ultralytics

In [11]:
from ultralytics import settings, YOLO

In [12]:
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/datasets',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [13]:
settings['datasets_dir'] = '/content/'

In [14]:
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [16]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

model = YOLO(model='/content/2_best.pt', task='detect') # task : detectt, segment, classify, pose
                                                # model : yolov8n.pt, yolov8s.pt, yolov8m.pt yolov8l.pt, yolov8x.pt

es = EarlyStopping(monitor='val_loss', patience=5)
mc = ModelCheckpoint('best.pt', monitor='val_loss', save_best_only=True)

In [17]:
model.train(model = '/content/2_best.pt',
            data='/content/ᅳ 5차_4일차/16조/16┴╢/16_team.yaml',
            epochs=200,
            verbose=True,
            patience=20,
            seed=2024,
            pretrained=True,
            # hsv_h=0,
            # hsv_s=0,
            # hsv_v=0,
            # translate=0,
            # scale=0,
            # fliplr=0,
            # mosaic=0,
            # erasing=0,
            # crop_fraction=0,
            )

Ultralytics YOLOv8.2.5 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=/content/2_best.pt, data=/content/ᅳ 5차_4일차/16조/16┴╢/16_team.yaml, epochs=200, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

100%|██████████| 755k/755k [00:00<00:00, 42.4MB/s]

Overriding model.yaml nc=13 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

 22        [15, 18, 21]  1   2118370  ultralytics.nn.modules.head.Detect           [6, [128, 256, 512]]          
Model summary: 225 layers, 11137922 parameters, 11137906 gradients, 28.7 GFLOPs

Transferred 349/355 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir runs/detect/train', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


100%|██████████| 6.23M/6.23M [00:00<00:00, 175MB/s]


...

                   all        229        229      0.925      0.862      0.919      0.712
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 61, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



81 epochs completed in 0.636 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.5 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 168 layers, 11127906 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.22it/s]


                   all        229        229      0.921      0.903      0.943      0.734
                   SIJ        229         60      0.992          1      0.995      0.904
                   LCH        229         40      0.974       0.95      0.968      0.829
                   KHS        229         60      0.921      0.777      0.926       0.64
                   YMS        229         29      0.954      0.897      0.953       0.68
                   YBH        229         40      0.764      0.893      0.875      0.616
Speed: 0.3ms preprocess, 2.8ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e4ba130c310>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [22]:
result = model.predict(source='/content/drive/MyDrive/program/miniPJT_5/DX_미니프로젝트 5차_4일차/img3',
                       conf=0.5,          # confidence score
                       iou=0.7,           # IoU가 0.7 이상 넘기면 같은 object를 가리키는 것이라고 판단
                       save=True,         # 예측된 이미지/동영상 저장 유무 설정
                       line_width=2       # 그려지는 박스이 두께 설정
                       )


image 1/4 /content/drive/MyDrive/program/miniPJT_5/DX_미니프로젝트 5차_4일차/img3/11.png: 288x640 2 LCHs, 1 KHS, 1 YMS, 1 YBH, 112.6ms
image 2/4 /content/drive/MyDrive/program/miniPJT_5/DX_미니프로젝트 5차_4일차/img3/20240430_144824.jpg: 640x480 1 LCH, 1 KHS, 11.4ms
image 3/4 /content/drive/MyDrive/program/miniPJT_5/DX_미니프로젝트 5차_4일차/img3/20240430_144844.jpg: 640x480 1 LCH, 10.9ms
image 4/4 /content/drive/MyDrive/program/miniPJT_5/DX_미니프로젝트 5차_4일차/img3/20240430_144903.jpg: 640x480 (no detections), 12.3ms
Speed: 2.9ms preprocess, 36.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/detect/train4


In [23]:
for r in result :
    boxes = r.boxes

### 모델 추가 학습하기